# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
class Keys:
    is_fetch_ok = "Is_Fetch_Ok"
    city_id     = "City_ID"
    city        = "City"
    cloudiness  = "Cloudiness"
    country     = "Country"
    date        = "Date"
    humidity    = "Humidity"
    lat         = "Lat"
    lng         = "Lng"
    max_temp    = "Max Temp"
    wind_speed  = "Wind Speed"
    hotel_name  = "Hotel Name"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_weather_metadata_path = "../output_data/cities.csv"

# Read the city data
city_weather_metadata = pd.read_csv(city_weather_metadata_path)
city_weather_df = pd.DataFrame(city_weather_metadata).astype({Keys.lat:        float,
                                                              Keys.lng:        float,
                                                              Keys.city_id:    int,
                                                              Keys.humidity:   int,
                                                              Keys.cloudiness: int,
                                                              Keys.date:       int})
city_weather_df.dropna(inplace = True) 
city_weather_df.set_index(Keys.city_id, inplace=True)
city_weather_df.head(10)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City_ID,,,,,,,,,
0,qaanaaq,23,GL,1635621182,99,84.02,-83.68,-11.04,11.39
1,busselton,28,AU,1635621182,90,-75.87,82.80,-54.83,18.61
2,vaini,100,TO,1635621183,87,-51.80,-171.79,44.15,26.06
3,tiksi,100,RU,1635621183,95,83.82,122.45,9.46,3.04
4,auki,61,SB,1635621183,66,-3.98,163.93,82.85,13.71
5,upernavik,100,GL,1635621183,98,84.32,-48.30,-5.06,8.84
6,ponta do sol,97,CV,1635621184,70,25.03,-35.63,77.13,8.34
7,ushuaia,100,AR,1635621184,82,-89.31,-45.99,-31.45,12.53
8,punta arenas,100,CL,1635621184,81,-58.71,-90.96,39.74,25.88


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#https://developers.google.com/maps/documentation/places/web-service/search-find-place

# Access maps with unique API key
gmaps.configure(api_key = g_key)

# Plot Heatmap
fig = gmaps.figure()

# Store latitude and longitude in locations
locations = city_weather_df[["Lat", "Lng"]]
humidity  = city_weather_df["Humidity"]

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, 
                                    weights = humidity,
                                    point_radius=10,
                                    max_intensity=99)
# Add layer
fig.add_layer(heatmap_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.

max_perfect_temp   = 82
min_perfect_temp   = 65
perfect_wind_speed = 10
perfect_cloudiness = 3

#NOTE: Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

perfect_cities_df = pd.DataFrame(city_weather_df.loc[(city_weather_df[Keys.max_temp]  <= max_perfect_temp) &  \
                                (city_weather_df[Keys.max_temp]   >= min_perfect_temp)                     &  \
                                (city_weather_df[Keys.wind_speed] <= perfect_wind_speed)                   &  \
                                (city_weather_df[Keys.cloudiness] <= perfect_cloudiness)])

print(f"The number of perfect cities is {len(perfect_cities_df)}")


The number of perfect cities is 23


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Store into variable named hotel_df
#Add a "Hotel Name" column to the DataFrame
hotel_df = perfect_cities_df.copy(deep=True)
hotel_df["Hotel Name"]=pd.Series(dtype=str)

In [8]:
#Set parameters to search for hotels with 5000 meters.
search_radius_meters = 5000

In [9]:
# NOTE: Do not change any of the code in this cell
#Source: https://towardsdatascience.com/how-to-use-the-google-places-api-for-location-analysis-and-more-17e48f8f25b1

latitudes     = hotel_df[Keys.lat]
longitudes    = hotel_df[Keys.lng]
country_codes = hotel_df[Keys.country]

lat_lng_ccs = zip(latitudes,
                  longitudes,
                  country_codes)

output_type = "json"
query       = "lodging"
search_type = "hotel"

for idx,data_row in hotel_df.iterrows():

    latitude     = data_row[Keys.lat]
    longitude    = data_row[Keys.lng]
    country_code = data_row[Keys.country]
    
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/{output_type}?query={query}&location={latitude},{longitude}&radius={search_radius_meters}&region={country_code}&type={search_type}&key={g_key}"    
    
    response    = requests.get(url)
    status_code = response.status_code  
    
    if status_code == 200:   
        
        json_obj = response.json()    
        hotel_name = json_obj["results"][0]["name"]
        
        #Source: https://stackoverflow.com/questions/23330654/update-a-dataframe-in-pandas-while-iterating-row-by-row
        
        hotel_df.at[idx,Keys.hotel_name] = hotel_name

    else:
        print("Ooops")
        
hotel_df      

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
City_ID,,,,,,,,,,
14,grand gaube,0,MU,1635621186,82,-16.64,60.26,76.69,6.46,USAG Humphreys Army Lodging
74,luganville,1,VU,1635621263,67,-15.20,167.68,81.93,7.90,Rarangi Tell homestay
237,samusu,0,WS,1635621425,67,-6.51,-162.23,79.23,6.58,Best Bet Lodging
250,de aar,0,ZA,1635621488,37,-30.90,24.35,65.61,8.32,Mieliefontein Karoo Guest Farm
262,marzuq,0,LY,1635621491,32,25.26,13.16,70.77,7.85,Air Force Inns (Pet Friendly TLF)
274,minab,0,IR,1635621494,26,26.59,57.71,65.88,2.53,شهرک ولیعصر
280,naze,3,JP,1635621496,66,27.56,129.19,73.94,9.22,日の出旅館
306,port hedland,0,AU,1635621563,88,-21.26,119.80,65.91,6.51,The Marble Bar Roadhouse and Travellers Rest
344,tiznit,0,MA,1635621573,79,30.02,-10.05,68.31,8.30,"Wagga d'Aglou Parapente, Chez Aziz"


In [10]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [11]:
# Add marker layer ontop of heat map
perfect_spots = hotel_df.reset_index()[[Keys.lat, Keys.lng]]
marker_layer = gmaps.marker_layer(perfect_spots, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))